# GPU-Accelerated Isogeometric Cloth Simulation on Watertight Surfaces

## Cloth Simulation: First Variations of Strain Measures

### Supplementary Material for Appendix B

**Rafael Cerqueira de Campos**  
Advisor: Prof. Dr. Afonso Paiva Neto  
Instituto de Ciencias Matematicas e de Computacao (ICMC)  
Universidade de Sao Paulo

---

*Following Appendix C of Kiendl et al. (2015) - Isogeometric Kirchhoff-Love shell formulations for general hyperelastic materials*

## 1. Setup and Definitions

In [ ]:
# Cartesian basis vectors
e1 = vector([1, 0, 0])
e2 = vector([0, 1, 0])
e3 = vector([0, 0, 1])
e = [e1, e2, e3]  # 0-indexed: e[0]=e1, e[1]=e2, e[2]=e3

# Symbolic tangent vectors
var('a1x a1y a1z a2x a2y a2z')
a1 = vector([a1x, a1y, a1z])
a2 = vector([a2x, a2y, a2z])

# Second parametric derivatives
var('a11x a11y a11z a12x a12y a12z a22x a22y a22z')
a11 = vector([a11x, a11y, a11z])
a12 = vector([a12x, a12y, a12z])
a22 = vector([a22x, a22y, a22z])

# Shape function derivatives
var('Na1 Na2 Na11 Na12 Na22')

# Helper to get shape function derivative by index
def Na(alpha, beta=None):
    """Get shape function derivative N^a_{,alpha} or N^a_{,alpha,beta}"""
    if beta is None:
        return Na1 if alpha == 1 else Na2
    else:
        if (alpha, beta) == (1, 1):
            return Na11
        elif (alpha, beta) == (2, 2):
            return Na22
        else:
            return Na12  # Na12 = Na21 by symmetry

print("Cartesian basis: e_1, e_2, e_3")
print("Tangent vectors: a_1, a_2 (symbolic)")
print("Second derivatives: a_{11}, a_{12}, a_{22} (symbolic)")
print("Shape function derivatives: N^a_{,1}, N^a_{,2}, N^a_{,11}, N^a_{,12}, N^a_{,22}")

## 2. Variation of Tangent Vectors (Eq. C.4)

The tangent vectors are $a_\alpha = \frac{\partial r}{\partial \theta^\alpha}$. Since $r = R + u$ and $R$ is fixed:

$$a_{\alpha,r} = \frac{\partial}{\partial u_r} \left( \frac{\partial u}{\partial \theta^\alpha} \right)$$

Using $u = \sum_a N^a u^a$, we get:

$$a_{\alpha,r} = N^a_{,\alpha} e_i$$

In [ ]:
def aAlphaVar(alpha, i):
    """Variation of tangent vector a_alpha with respect to u_r in direction i"""
    return Na(alpha) * e[i-1]  # i is 1-indexed

# Evaluate for each component
print("a_{1,r} for i=1 (x-direction):", aAlphaVar(1, 1))
print("a_{1,r} for i=2 (y-direction):", aAlphaVar(1, 2))
print("a_{1,r} for i=3 (z-direction):", aAlphaVar(1, 3))

print("\nSince Na[alpha] is independent of u_s, we have a_{alpha,rs} = 0")

## 3. Variation of Second Derivatives (Eq. C.6)

Similarly, for the second parametric derivative:

$$a_{\alpha,\beta,r} = N^a_{,\alpha\beta} e_i$$

In [ ]:
def aAlphaBetaVar(alpha, beta, i):
    """Variation of second derivative a_{alpha,beta} with respect to u_r in direction i"""
    return Na(alpha, beta) * e[i-1]

print("a_{1,1,r} for i=1:", aAlphaBetaVar(1, 1, 1))
print("a_{1,2,r} for i=3:", aAlphaBetaVar(1, 2, 3))

## 4. Membrane Strain Variation (Eqs. C.8, C.21)

The membrane strain is:

$$\varepsilon_{\alpha\beta} = \frac{1}{2}(a_{\alpha\beta} - A_{\alpha\beta})$$

where the metric coefficient is:

$$a_{\alpha\beta} = a_\alpha \cdot a_\beta$$

Applying the product rule to the metric coefficient variation:

In [ ]:
print("a_{αβ,r} = a_{α,r} · a_β + a_α · a_{β,r}")
print("a_{αβ,r} = (N^a_{,α} e_i) · a_β + a_α · (N^a_{,β} e_i)")
print("a_{αβ,r} = N^a_{,α} (e_i · a_β) + N^a_{,β} (e_i · a_α)")

In [ ]:
def metricVar(alpha, beta, i, aAlpha, aBeta):
    """Variation of metric coefficient a_{alpha,beta}"""
    return Na(alpha) * (e[i-1].dot_product(aBeta)) + Na(beta) * (e[i-1].dot_product(aAlpha))

def epsilonVar(alpha, beta, i, aAlpha, aBeta):
    """Membrane strain variation (half the metric variation)"""
    return (1/2) * metricVar(alpha, beta, i, aAlpha, aBeta)

# Example: epsilon_{11,r} for i=1
print("ε_{11,r} for i=1:")
print("  = (1/2)[N^a_{,1} (e_1 · a_1) + N^a_{,1} (e_1 · a_1)]")
print("  = (1/2)[2 N^a_{,1} (a_1)_x]")
print("  = N^a_{,1} (a_1)_x")
print("  =", epsilonVar(1, 1, 1, a1, a1))

print("\nε_{12,r} for i=1:", epsilonVar(1, 2, 1, a1, a2))

## 5. Unit Normal: Auxiliary Variables (Eqs. C.10-C.12)

The unit normal involves a cross product and normalization. We decompose it:

$$\tilde{a}_3 = a_1 \times a_2$$ (unnormalized normal)

$$\bar{a}_3 = \|\tilde{a}_3\|$$ (magnitude = Jacobian)

$$a_3 = \frac{\tilde{a}_3}{\bar{a}_3}$$ (unit normal)

In [ ]:
# Compute unnormalized normal
a3Tilde = a1.cross_product(a2)
print("a~_3 = a_1 × a_2 =", a3Tilde)

# Compute magnitude (symbolically)
a3Bar = sqrt(a3Tilde.dot_product(a3Tilde))
print("\na̅_3 = ||a~_3|| = sqrt(a~_3 · a~_3)")

# Compute unit normal
a3 = a3Tilde / a3Bar
print("\na_3 = a~_3 / a̅_3")

## 6. Variation of Auxiliary Variables (Eqs. C.13, C.15)

We need variations of the auxiliary variables to get the normal variation.

From Eq. C.13, using the product rule for cross products:

$$\tilde{a}_{3,r} = a_{1,r} \times a_2 + a_1 \times a_{2,r}$$

In [ ]:
print("a~_{3,r} = a_{1,r} × a_2 + a_1 × a_{2,r}")
print("        = (N^a_{,1} e_i) × a_2 + a_1 × (N^a_{,2} e_i)")

def a3TildeVar(i):
    """Variation of unnormalized normal"""
    return aAlphaVar(1, i).cross_product(a2) + a1.cross_product(aAlphaVar(2, i))

# Evaluate for i=1
a3TildeVar1 = a3TildeVar(1)
print("\na~_{3,r} for i=1:", a3TildeVar1.simplify_full())

From Eq. C.15, using chain rule on the magnitude:

$$\bar{a}_{3,r} = \frac{\partial}{\partial u_r} \sqrt{\tilde{a}_3 \cdot \tilde{a}_3} = \frac{\tilde{a}_3 \cdot \tilde{a}_{3,r}}{\bar{a}_3} = a_3 \cdot \tilde{a}_{3,r}$$

In [ ]:
def a3BarVar(i):
    """Variation of magnitude (Jacobian)"""
    return a3.dot_product(a3TildeVar(i))

a3BarVar1 = a3BarVar(1).simplify_full()
print("a̅_{3,r} for i=1:", a3BarVar1)

## 7. Variation of Unit Normal (Eq. C.17)

Now we apply the quotient rule to $a_3 = \frac{\tilde{a}_3}{\bar{a}_3}$:

$$a_{3,r} = \frac{\tilde{a}_{3,r} \bar{a}_3 - \tilde{a}_3 \bar{a}_{3,r}}{\bar{a}_3^2}$$

Simplifying:

$$a_{3,r} = \frac{1}{\bar{a}_3} \left( \tilde{a}_{3,r} - \bar{a}_{3,r} a_3 \right)$$

In [ ]:
def a3Var(i):
    """Variation of unit normal"""
    return (1/a3Bar) * (a3TildeVar(i) - a3BarVar(i) * a3)

a3Var1 = a3Var(1).simplify_full()
print("a_{3,r} for i=1:", a3Var1)

# Verify: a3Var should be perpendicular to a3
perpCheck = a3.dot_product(a3Var1).simplify_full()
print("\nVerification: a_3 · a_{3,r} =", perpCheck, "(should be 0)")
print("This confirms that the normal can only rotate, not change magnitude.")

## 8. Bending Strain Variation (Eqs. C.19, C.23)

The bending strain is:

$$\kappa_{\alpha\beta} = B_{\alpha\beta} - b_{\alpha\beta}$$

where the curvature coefficient is:

$$b_{\alpha\beta} = a_{\alpha,\beta} \cdot a_3$$

Applying product rule (Eq. C.19):

$$b_{\alpha\beta,r} = a_{\alpha,\beta,r} \cdot a_3 + a_{\alpha,\beta} \cdot a_{3,r}$$

And from Eq. C.23:

$$\kappa_{\alpha\beta,r} = -b_{\alpha\beta,r}$$

Assuming the reference curvature $B_{\alpha\beta}$ is fixed with respect to the displacement variation.

In [ ]:
def bAlphaBetaVar(alpha, beta, i, aab):
    """Variation of curvature coefficient b_{alpha,beta}"""
    return aAlphaBetaVar(alpha, beta, i).dot_product(a3) + aab.dot_product(a3Var(i))

def kappaVar(alpha, beta, i, aab):
    """Bending strain variation (negative of curvature variation)"""
    return -bAlphaBetaVar(alpha, beta, i, aab)

# Show the two terms separately for kappa_{11,r} with i=1
term1 = aAlphaBetaVar(1, 1, 1).dot_product(a3)
term2 = a11.dot_product(a3Var(1))

print("For κ_{11,r} with i=1:\n")
print("  Term I (from a_{α,β,r} · a_3):", term1.simplify_full())
print("  Term II (from a_{α,β} · a_{3,r}):", term2.simplify_full())
print("\n  κ_{11,r} = -(Term I + Term II) =", kappaVar(1, 1, 1, a11).simplify_full())

## 9. Numerical Verification

In [ ]:
# Define numerical values
numRules = {
    a1x: 1.0, a1y: 0.1, a1z: 0.0,
    a2x: 0.1, a2y: 1.0, a2z: 0.0,
    a11x: 0.0, a11y: 0.0, a11z: 0.1,
    Na1: 0.25, Na2: 0.15, Na11: 0.05
}

# Evaluate unit normal numerically
a3Num = vector([comp.subs(numRules) for comp in a3])
print("Unit normal:", a3Num)
print("||a_3|| =", a3Num.norm().n(), "(should be 1)")

# Evaluate normal variation
a3Var1_expr = a3Var(1)
a3Var1Num = vector([comp.subs(numRules).n() for comp in a3Var1_expr])
print("\na_{3,r} for i=1:", a3Var1Num)

# Verify orthogonality
dot_check = sum(a3Num[i] * a3Var1Num[i] for i in range(3)).n()
print("a_3 · a_{3,r} =", dot_check, "(should be 0)")

# Evaluate bending strain variation
kappa11Var1_expr = kappaVar(1, 1, 1, a11)
kappa11Var1Num = kappa11Var1_expr.subs(numRules).n()
print("\nκ_{11,r} for i=1:", kappa11Var1Num)

---

## Summary

This notebook derives the first variations of:

1. **Tangent vectors**: $a_{\alpha,r} = N^a_{,\alpha} e_i$

2. **Membrane strain**: $\varepsilon_{\alpha\beta,r} = \frac{1}{2}\left[ N^a_{,\alpha}(e_i \cdot a_\beta) + N^a_{,\beta}(e_i \cdot a_\alpha) \right]$

3. **Unit normal**: $a_{3,r} = \frac{1}{\bar{a}_3}\left( \tilde{a}_{3,r} - \bar{a}_{3,r} a_3 \right)$

4. **Bending strain**: $\kappa_{\alpha\beta,r} = -\left( a_{\alpha,\beta,r} \cdot a_3 + a_{\alpha,\beta} \cdot a_{3,r} \right)$

These variations are essential for computing the internal force vector and tangent stiffness matrix in the finite element formulation of Kirchhoff-Love shells.